<a href="https://colab.research.google.com/github/etarazonav/650044-ABD-ULIMA/blob/main/Notebooks/ABB_SparkSQ2_Consultas_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Universidad_de_Lima_logo.png/220px-Universidad_de_Lima_logo.png"  width="120" /> Parte 2: Uso de consultas SQL

**Profesor:** Enver G. Tarazona Vargas <br>
**Curso:** Analítica con Big Data <br>
**FACULTAD DE INGENIERÍA - CARRERA DE INGENIERÍA DE SISTEMAS**<br>


La documentación sobre el API de SQL en Spark es: https://spark.apache.org/docs/latest/sql-ref.html

In [ ]:
!pip install -q pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
# Carga de archivos
!wget -q https://raw.githubusercontent.com/etarazonav/650044-ABD-ULIMA/refs/heads/main/Datos/personas.csv
!wget -q https://raw.githubusercontent.com/etarazonav/650044-ABD-ULIMA/refs/heads/main/Datos/poema.txt


## Ejemplo 1: A partir de un DataFrame

**Creación de un DataFrame**

In [ ]:
df = spark.read.load("personas.csv",
                     format="csv", sep=";", inferSchema=True, header=True)
df.show(2)

**Vista: "Creación de una tabla"**

Para poder realizar consultas ("queries") usando SQL sobre un DataFrame, primero se debe crear una vista temporal del DataFrame usando `createOrReplaceTempView`. Luego sobre esta vista temporal se realiza las consultas usando SQL.

In [ ]:
# Crear una "tabla" llamada "tabla1" con los datos del dataframe df
df.createOrReplaceTempView('tabla1')

**Ejemplos de consultas:**

Se puede realizar consultas de tipo SQL sobre la vista temporal creada, que en este caso se llama `tabla1`. El resultado es un DataFrame de PySpark.

In [ ]:
spark.sql("SELECT * FROM tabla1")

Para visualizar el resultado se puede colocar directamente `show`, o se puede almacenar en una variable (DataFrame) y visualizar dicho DataFrame usando `show`.

In [ ]:
df2 = spark.sql("SELECT * FROM tabla1")
df2.show()

In [ ]:
spark.sql("SELECT * FROM tabla1 ORDER BY edad DESC LIMIT 2").show()

In [ ]:
spark.sql("SELECT * FROM tabla1 WHERE edad<26").show()

In [ ]:
spark.sql("SELECT * FROM tabla1 WHERE nombre LIKE 'Jorge'").show()

In [ ]:
spark.sql("SELECT count(*) AS Elementos from tabla1").show()

**Creación de UDF (User Defined Function)**

In [ ]:
from pyspark.sql.functions import pandas_udf
import pandas as pd

In [ ]:
@pandas_udf("integer")
def suma_cien(s: pd.Series) -> pd.Series:
    return s + 100

spark.udf.register("suma_cien", suma_cien)

Uso del UDF para una consulta sobre los datos usando SQL.

In [ ]:
spark.sql("SELECT nombre, suma_cien(edad) AS edad100 FROM tabla1").show()

In [ ]:
from pyspark.sql.functions import expr

df.selectExpr('suma_cien(edad)').show()

## Ejemplo 2: A partir de un RDD

Si la entrada de datos utiliza un RDD, primero se debe convertir a un DataFrame y luego recién a una vista temporal para poder realizar consultas SQL sobre dicha vista temporal.

In [ ]:
sc = spark.sparkContext

# Creación de un RDD a partir de un archivo de texto
rdd = sc.textFile("poema.txt")
rdd.take(3)

In [ ]:
# Se importa re para usar expresiones regulares
import re

# Contar palabras usando el esquema MapReduce
rdd2 = rdd.flatMap(lambda x: x.lower().split())\
          .map(lambda x: (re.sub(r'\W+', '', x), 1)) \
          .reduceByKey(lambda x,y: x+y)

rdd2.take(10)

### Uso de Expresiones Regulares con la transformación `map`
1. **`map(lambda x: ...)`**:
   - `map` es una operación en Spark que transforma cada elemento del RDD aplicando la función dada (en este caso, la función anónima `lambda x: ...`). Aquí, cada palabra que viene de la operación anterior en el RDD será el valor de `x`.

2. **`(re.sub(r'\W+', '', x), 1)`**:
   - **`re.sub(r'\W+', '', x)`**: Esto usa la función `re.sub` de la biblioteca `re` (que maneja expresiones regulares en Python). Esta función toma tres argumentos:
     1. **`r'\W+'`**: Esta es la expresión regular. Desglosemosla:
        - `\W` significa cualquier carácter que *no* sea una letra, número o guion bajo.
        - El `+` indica que esta búsqueda será para una o más de esas coincidencias consecutivas.
     2. **`''`**: Es el valor de reemplazo, que en este caso es una cadena vacía. Significa que todo lo que coincida con la expresión regular se eliminará.
     3. **`x`**: Es el texto o palabra que se está limpiando.

     Por lo tanto, la función `re.sub(r'\W+', '', x)` reemplaza cualquier secuencia de caracteres no alfanuméricos (espacios, puntuación, símbolos) en la palabra `x` con una cadena vacía, esencialmente eliminando estos caracteres y dejando solo letras y números.

   - **Ejemplo**:
     - Si `x = 'Hola, mundo!'`, la expresión **`re.sub(r'\W+', '', x)`** convertirá la palabra en `'Holamundo'` (eliminando la coma y el signo de exclamación).
   
3. **`, 1`**:
   - Después de limpiar la palabra con `re.sub`, se devuelve un par `(palabra_limpia, 1)`.
   - El `1` es simplemente un marcador que más adelante se utilizará para contar cuántas veces aparece cada palabra.

### Ejemplo paso a paso:

- Supongamos que la palabra original es `"¡Hola!"`.
- El **`re.sub(r'\W+', '', "¡Hola!")`** elimina todos los caracteres no alfanuméricos, devolviendo `"Hola"`.
- El resultado de la expresión **`map(lambda x: (re.sub(r'\W+', '', x), 1))`** sería el par **`('Hola', 1)`**.

### Resumen de la expresión:

- **`re.sub(r'\W+', '', x)`**: Limpia cada palabra eliminando todos los caracteres que no sean letras o números.
- **`map(lambda x: (re.sub(r'\W+', '', x), 1))`**: Toma cada palabra limpia y la convierte en un par clave-valor donde la clave es la palabra y el valor es `1`, preparándolo para el conteo de palabras más adelante.



In [ ]:
from pyspark.sql import Row

# Crear un RDD con filas de tipo "Row"
rdd3 = rdd2.map(lambda x: Row(palabra=x[0], contador=x[1], longitud=len(x[0])))
rdd3.take(3)

In [ ]:
# Crear un DataFrame a partir del RDD
df = spark.createDataFrame(rdd3)

df.show(4)

In [ ]:
# Crear una vista temporal
df.createOrReplaceTempView("tabla2")

# Consultas SQL en la tabla
spark.sql("SELECT palabra, longitud FROM tabla2 WHERE contador>=3 ORDER BY longitud DESC").show(10)